### Importing Libraries

In [1]:
import os
import json
import boto3
import re

import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

### Creating config

In [2]:
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_session.region_name
role = get_execution_role()

### Setting Path to S3

In [3]:
S3_BUCKET = "sagemaker-taxi-fare"

TRAIN_FOLDER  = r"train/"
TEST_FOLDER  = r"test/"
MODEL_FOLDER  = r"model/"

TRAIN_DATA_URI = r"s3://" + S3_BUCKET + r"/" + TRAIN_FOLDER
TEST_DATA_URI = r"s3://" + S3_BUCKET + r"/" + TEST_FOLDER

### Prepare files required to build the containers

In [4]:
REPOSITORY = "taxi-fare"

In [5]:
!mkdir build
!cp Dockerfile build/
!cp requirements.txt build/
!mkdir build/src
!cp src/* build/src/

In [6]:
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.session.Session().region_name

os.environ["AWS_ACCOUNT_ID"] = account_id
os.environ["AWS_REGION"] = region
os.environ["REPO_NAME"] = REPOSITORY

In [ ]:
%%sh

cd build
docker build -t $REPO_NAME .
aws ecr get-login-password --region $AWS_REGION | docker login --username AWS --password-stdin $AWS_ACCOUNT_ID.dkr.ecr.$AWS_REGION.amazonaws.com
aws ecr create-repository --repository-name $REPO_NAME
docker tag $REPO_NAME:latest $AWS_ACCOUNT_ID.dkr.ecr.$AWS_REGION.amazonaws.com/$REPO_NAME:latest
docker push $AWS_ACCOUNT_ID.dkr.ecr.$AWS_REGION.amazonaws.com/$REPO_NAME:latest
cd ..
rm -r build/

### Training Estimator

In [8]:
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY}:latest"

In [9]:
estimator = Estimator(
    image_uri=image_uri,
    role=get_execution_role(),
    base_job_name="taxi-fare-v1",
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters = {'cv_folds': 3, 'verbose': 1}
)

In [10]:
estimator.fit({
    "train": TRAIN_DATA_URI,
    "test": TEST_DATA_URI
})

2021-11-18 19:53:05 Starting - Starting the training job...
2021-11-18 19:53:14 Starting - Launching requested ML instancesProfilerReport-1637265184: InProgress
......
2021-11-18 19:54:33 Starting - Preparing the instances for training.........
2021-11-18 19:56:01 Downloading - Downloading input data
2021-11-18 19:56:01 Training - Downloading the training image...
2021-11-18 19:56:34 Training - Training image download completed. Training in progress..[2021-11-18 19:56:34,799] INFO - TRAINING STARTED...
[2021-11-18 19:56:34,799] INFO - PARAMETERS LOADED!
[2021-11-18 19:56:35,009] INFO - DATA LOADED!
[2021-11-18 19:56:53,973] INFO - FEATURES ENGINEERED!
[2021-11-18 19:56:53,977] INFO - FEATURES AND TARGET VALUES DEFINED!
[2021-11-18 19:56:53,978] INFO - MODEL ARTIFACTS CREATED!
[2021-11-18 19:56:53,978] INFO - TRAINING MODEL...
[2021-11-18 19:56:53,984] INFO - [GRADIENT_BOOSTING]
[2021-11-18 20:05:53,062] INFO - #011Best parameters:
- learning_rate = 0.1000
- max_depth = 6.0000
- n_estim

### Deploying Estimator

In [11]:
ENDPOINT_NAME = "taxi-fare-nyc"
runtime = boto3.client('runtime.sagemaker')

In [12]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=ENDPOINT_NAME
)

----!

In [13]:
input_data = {
    "taxi_ride_id": 1,
    "pickup_datetime": "2010-04-19 08:17:56 UTC",
    "pickup_longitude": -73.992365,
    "pickup_latitude": 40.730521,
    "dropoff_longitude": -73.975499,
    "dropoff_latitude": 40.744746,
    "passenger_count": 1,
}

In [14]:
response  = runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=json.dumps(input_data)
)

In [15]:
result = json.loads(response['Body'].read().decode())
result

{'taxi_fare_amount': 7.51, 'taxi_ride_id': 1}

Cleaning up endpoint once we are done with it

In [1]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint)